# Version 3.1
In KMC-CH version 2 series, the code is rebuilt as compared to version 1.  
Check update in this link: http://www.evernote.com/l/AjzZYZtFa35AV7hPiPMWIsesG9tjAZAgl_4/  

For update since version 2.0:  
1. add 'write to csv' function
2. post analysis: dilute the time course when writing to csv files
3. Change enzyme concentration to 8 nM (v2.3)
4. dl[]: Cancel the list of rate matrix and lattice matrix, in order to release memory.
5. func_gamma: cut the scanning range to the vicinity of the excuted point
6. Add the progress output to KMC loop, using tqdm: https://pypi.python.org/pypi/tqdm#ipython-jupyter-integration  
7. Add a leakage from E1 to bridge
8. adsorption on bridge is included
9. scan the whole lattice surface in each KMC step

In [ ]:
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import animation, rc
from IPython.display import HTML
from JSAnimation.IPython_display import display_animation
import copy

#%precision %.4g
%matplotlib inline

### KMC Settings

samp = "sample_index" # set the name of this simulation or sample

lat_hop=4 # number of hopping sites
prl=100    # number of parallel cascades
st=10**7  # number of kMC steps
rep=2    # number of repeating KMC loops

In [ ]:
samp = "sample_index" # set the name of this simulation or sample

prl=100    # number of parallel cascades
st=10**7  # number of kMC steps
rep=5    # number of repeating KMC loops

In [ ]:
N_hop = [number_of_hop_site, 3] # number of hopping sites

xE1 = 0            # index of E1
xE2 = N_hop[0]+1    # index of E2
xE3 = sum(N_hop)+2  # index of E3
dim = sum(N_hop)+3  # length of lattice

In [ ]:
T=310 # temperature in K

c_enzyme= 8E-9 # concentration of channeling cascade in mol/L
vol=(prl/sc.N_A)/c_enzyme # volume of the system in L

c_sub=2 # concentration of substrate for E1 in mol/L
N_sub=c_sub*vol*sc.N_A # number of bulk substrate for E1

### Rate constant from MD and Expt

In [ ]:
k_cat     = np.array([0.7, 6.2, 6.2])          # turnover frequency for enzyme 1 2 3, in molecules/s
KM       = np.array([10**(-5), 1/(1.85*10**5), 1/(1.85*10**5)])      # Michaelis Constant for each enzyme
N_KM     = KM*sc.N_A*vol                                             # corresponding molecule number

k_des_sub = k_cat*0.1                          # substrate desorption rate on each enzyme
k_ads_sub = (k_cat+k_des_sub)/KM

k_hop_pdt = k_cat*100                     # product hopping rate from each enzyme to bridge
k_des_pdt = k_hop_pdt[:2]*np.array([0, 0])   # product desorption rate on each enzyme
k_ads_pdt = np.array([0, 0, 0])                  # product adsorption rate on each enzyme

k2k_brbr_hop2des = np.array([khop__kdes, khop__kdes])          # the rate constant ratio of hopping to desorption, on bridge
k2k_brbr_ads2des = np.array([kads__kdes,kads__kdes])        # the rate constant ratio of adsorption to desorption, on bridge
k2k_brE2_hop2des = np.array([khop_BrE2__kdes, khop_BrE2__kdes])          # the rate constant ratio of hopping to desorption, from bridge to enzyme

k_hop_itr = k_cat[:2]*100          # hopping rate of intermediate on bridge
k_des_itr = k_hop_itr / k2k_brbr_hop2des        # intermediate desorption rate on bridge
k_trv_itr = k_des_itr * k2k_brE2_hop2des       # intermediate hopping rate from bridge to enzyme, traverse
k_ads_itr = k_des_itr * k2k_brbr_ads2des          # adsorption rate of intermediate on bridge

k_inf = k_cat[0]*10**16 # set a infinit rate constant

### The switchs

In [ ]:
## Turn this on if you want to turn off the leaking from E1 to bridge
k_des_pdt[0]=0
k_des_pdt[1]=0

## Turn this on if you want to run symmetric leakage on each enzyme
k_des_pdt=k_hop_pdt[:2]/k2k_brE2

## Turn this on if you want to turn off the leaking from bridge to E2,E3
k_trv_itr[0]=k_hop_itr[0]
k_trv_itr[1]=k_hop_itr[1]

## Turn this on if you want to run free standing system
k_des_pdt[0]=k_inf*1
k_des_pdt[1]=k_inf*1

k_ads_itr[0]=0
k_ads_itr[1]=0

k_hop_pdt[0]=0
k_hop_pdt[1]=0

In [ ]:
## Turn this on if you want to run two step reaction
k_cat[2]=0
k_des_sub[2]=0
k_ads_sub[2]=0
k_des_itr[1]=0
k_ads_itr[1]=0
k_hop_itr[1]=0
k_trv_itr[1]=0
k_hop_pdt[1]=0
k_des_pdt[1]=k_inf*1

### Rate constant matrix

In [ ]:
'''Define the 'rate constant matrix', which will be used to assign value to rate matrix
row is event
column is the type of site: e.g.,0. E1; 1. bridge +1; 2. other bridge; 3. bridge -1 4. E2
'''
RC_mat_ori = [[k_ads_sub[0],k_ads_itr[0],k_ads_itr[0],k_ads_itr[0],k_ads_sub[1],k_ads_itr[1],k_ads_itr[1],k_ads_itr[1],k_ads_sub[2]], # ads of sub
              [k_des_sub[0],           0,           0,           0,k_des_sub[1],           0,           0,           0,k_des_sub[2]], # des of sub
              [    k_cat[0],           0,           0,           0,    k_cat[1],           0,           0,           0,    k_cat[2]], # reaction of sub
              [k_des_pdt[0],k_des_itr[0],k_des_itr[0],k_des_itr[0],k_des_pdt[1],k_des_itr[1],k_des_itr[1],k_des_itr[1],           0], # des of pdt
              [k_hop_pdt[0],k_hop_itr[0],k_hop_itr[0],k_trv_itr[0],k_hop_pdt[1],k_hop_itr[1],k_hop_itr[1],k_trv_itr[1],           0], # right hop of inter
              [           0,           0,k_hop_itr[0],k_hop_itr[0],           0,           0,k_hop_itr[1],k_hop_itr[1],           0]  # left hop of inter
             ]
RC_mat_ori = np.array(RC_mat_ori)

# duplicate the middle bridge site, making the rate constant matix the same dimension with lattice surface
RC_mat=np.array([r[0:2].tolist()+
                 [r[2]]*(N_hop[0]-2)+
                 r[3:6].tolist()+
                 [r[6]]*(N_hop[1]-2)+
                 r[7:9].tolist() for r in RC_mat_ori])

RC_mat[0][0]=RC_mat[0][0]*c_sub # assign concentration value to the adsorption rate on E1

#### Preview 'Rate constant matrix'
print('Order of magnitude for rate constant matrix: (-inf is from zero rate)')
print(RC_mat.shape)
print(np.round(np.log10(RC_mat)))

#### Define a matrix of the beginning and ending points for lattice surface scan

In [ ]:
sb_0=np.array([[0,-1,-1,-1,-1], # adsorption sub
               [0,-1,-1,-1,-1], # desorption sub
               [0,-1,-1,-1,-1], # reaction sub
               [0,-1,-1,-1,-1], # desorption pdt
               [0,-1,-1,-1,-1], # right hop
               [0,-1,-2,-2,-2]])# left hop

se_0=np.array([[2,2,2,2,1], # adsorption sub
               [2,2,2,2,1], # desorption sub
               [2,2,2,2,1], # reaction
               [2,2,2,2,1], # desorption pdt
               [3,3,3,2,1], # right hop
               [2,2,2,2,1]])# left hop

sb=np.array([r[0:2].tolist()+[r[2]]*(sum(N_hop)-1)+r[3:5].tolist() for r in sb_0])
se=np.array([r[0:2].tolist()+[r[2]]*(sum(N_hop)-1)+r[3:5].tolist() for r in se_0])
print(sb.shape)
print(sb)
print(se)

#### Brief summary of the system

In [ ]:
print('Ratio of actual TOF and k_cat on E1:')
print(c_sub/(c_sub+KM[0]))

In [ ]:
print('Number of cascade:%i'%prl)
print('Box volume: %.2f e-15 Liter'%(10**15*vol))
print('KM on E1 and E2 are %.3f, %.3f, %.3f mM' %(1000*KM[0],1000*KM[1],1000*KM[2]))
print('[I] for E2,E3 to reach E1 rate: %.2f, %.2f uM' %(10**6*k_cat[0]*KM[1]/(k_cat[1]-k_cat[0]),10**6*k_cat[0]*KM[2]/(k_cat[2]-k_cat[0])))
print('Analytical lag time for free standing system: %.1f sec' %(k_cat[0]*KM[1]/(k_cat[1]-k_cat[0])/(c_enzyme*k_cat[0])))

In [ ]:
k1_to_k2=100
dE=np.log(k1_to_k2)*T*sc.R/1000
print('If rate ratio k1/k2 is %i, the energy difference is %.2f kJ/mol' %(k1_to_k2,dE) )

### Occurance matrix (y&n) and generation function: *<font color=blue>func_occur</font>*

In [ ]:
#  Makge the matrix that judge the occurance of each event, based the lattice matrix
# 1 is yes, zeor or negative is no
def func_occur_assist(nhop): # input nhop should be a 2 element array

    # 1.ads sub
    ads_sub = np.zeros(shape=(dim,dim),dtype=int)        

    # 2.des sub
    des_sub = np.zeros(shape=(dim,dim),dtype=int)
    des_sub[xE1,xE1], des_sub[xE2,xE2], des_sub[xE3,xE3] = -1,1,-1

    # 3.reaction sub
    rct_sub = np.zeros(shape=(dim,dim),dtype=int)
    rct_sub[xE1,xE1], rct_sub[xE2,xE2], rct_sub[xE3,xE3] = -1,1,-1

    # 4.des pdt/inter
    des_pdt            = np.diag(np.ones(shape=dim,dtype=int), 0)
    des_pdt[xE2:,xE2:] = des_pdt[xE2:,xE2:]*(-1)
    des_pdt[xE1,xE1], des_pdt[xE2,xE2], des_pdt[xE3,xE3] = 1,-1,1

    # 5.hop right
    right = np.diag(-np.ones(shape=(dim-1),dtype=int), -1) + np.diag(np.ones(shape=dim,dtype=int), 0)
    right[:,(-1)] = 0

    # 6.hop left
    left  = np.diag(-np.ones(shape=(dim-1),dtype=int), 1) + np.diag(np.ones(shape=dim,dtype=int), 0)
    left[:,(xE1,(xE1+1),xE2,(xE2+1),xE3)] = 0

    return np.array([ads_sub,des_sub,rct_sub,des_pdt,right,left])

# make the occur assist matrix for this system
assist_mat = func_occur_assist(nhop=N_hop)
print(type(assist_mat))
print(assist_mat)

In [ ]:
def func_occur(lat_1D,ast_mat): # lat_1D is a 1d array of a specific cascade
    
    if len(lat_1D)!=ast_mat.shape[-1]:
        print('1D lattice and hopping site does not match!!!')
    
    else:
        # 1.ads sub
        ocr_ads_sub = abs(abs(lat_1D)-1)       

        # 2.des sub
        ocr_des_sub = np.dot(lat_1D,ast_mat[1]).clip(min=0)
        
        # 3.reaction sub
        ocr_rct_sub = np.dot(lat_1D,ast_mat[2]).clip(min=0)

        # 4.des pdt/inter
        ocr_des_pdt = np.dot(lat_1D,ast_mat[3]).clip(min=0)

        # 5.hop right
        ocr_right = ocr_des_pdt*np.dot(abs(lat_1D),ast_mat[4]).clip(min=0)

        # 6.hop left
        ocr_left = ocr_des_pdt*np.dot(abs(lat_1D),ast_mat[5]).clip(min=0)
        
        return np.array([ocr_ads_sub,ocr_des_sub,ocr_rct_sub,ocr_des_pdt,ocr_right,ocr_left])
        
# Do a quick test
lat_test = np.random.randint(3, size=(1, dim))-1
ocr_test = np.array([func_occur(lat_1D=l,ast_mat=assist_mat) for l in lat_test])

import pandas as pd
df=pd.DataFrame(lat_test)
s=['ads_sub','des_sub','rct_sub','des_pdt','right','left']
for i in np.arange(6):
    df.loc[s[i]]=ocr_test[0][i]
df

###  Rate calculation function: *<font color=blue>func_gamma</font>*

In [ ]:
def func_gamma(lat_in,conc_in,gamma_in,index,scan):
    
    gamma_in = copy.deepcopy(gamma_in)
    
    k,j,i=index # Get the site executed last step    
    
    if scan=='whole':     # scan=whole, scan the whole lattice surface; scan=vicinity, scan the changed site of lattice surface
        scan_j = np.arange(lat_in.shape[0])
        b=0
        e=lat_in.shape[1]        
    elif scan=='vicinity':
        scan_j = [j]
        b=i+sb[k,i]
        e=i+se[k,i]
    else:
        print('wrong input of scan')
      
    for jj in scan_j:# update the rate arount the event site
        gamma_in[:,jj,b:e] = R_mat[:,b:e] * func_occur(lat_1D=lat_in[jj],ast_mat=assist_mat)[:,b:e]
   
    if k in [0,1,3]: # if the bulk concentration change, update the ads event for all sites
        R_mat[0][1:(xE2+1)] = RC_mat[0][1:(xE2+1)]*conc_in[0] # involve the bulk concentration to bridge and active sites
        R_mat[0][(xE2+1):]  = RC_mat[0][(xE2+1):]*conc_in[1]    
        
        for jj in np.arange(lat_in.shape[0]):
            gamma_in[0,jj,1:] = R_mat[0,1:] * abs(abs(lat_in[jj,1:])-1)
    
    return gamma_in

#### Quick test on func_gamma
lat_test = np.random.randint(3, size=(1, 11))-1
gamma_test = np.zeros((6, lat_test.shape[0],lat_test.shape[1]))
print(lat_test)
gamma_test = func_gamma(lat_in = lat_test,
                        conc_in = [10**(-5),10**(-5)],
                        RC_in   = RC_mat,
                        gamma_in = gamma_test,
                        index = (4,3,5),
                        scan='whole')
print(np.round(np.log10(gamma_test))) # show the order of magnitude of the rate matrix

### Execution function: *<font color=blue>func_ext</font>*

In [ ]:
def func_ext(lat_in,Nbk_in,index):    
    lat_in=copy.deepcopy(lat_in)
    Nbk_in=copy.deepcopy(Nbk_in) # 3 element array with the number of bulk intermediate
    k,j,i = index
    
    if k==0:     # substrate adsorption
        if i==xE1 and lat_in[j,i]==0:
            lat_in[j,i]=-1
        elif i in np.arange(1,(xE2+1)) and lat_in[j,i]==0:
            lat_in[j,i]=1
            Nbk_in[0] -= 1
        elif i in np.arange(xE2+1,xE3+1) and lat_in[j,i]==0:
            lat_in[j,i]=-1
            Nbk_in[1] -= 1        
        else:
            print('Error k=0: wrong index or substrate adsorption is not allowed on bridge.')
            
    elif k==1:     # substrate desorption        
        if i==xE1 and lat_in[j,i]==-1:
            lat_in[j,i]=0
        elif i==xE2 and lat_in[j,i]==1:
            lat_in[j,i]=0
            Nbk_in[0]+=1
        elif i==xE3 and lat_in[j,i]==-1:
            lat_in[j,i]=0
            Nbk_in[1]+=1            
        else:
            print('Error k=1: wrong index or substrate desorption not allowed on bridge')    
    
    elif k==2:     # reaction        
        if i==xE1 and lat_in[j,i]==-1:
            lat_in[j,i]*=(-1)
        elif i==xE2 and lat_in[j,i]==1:
            lat_in[j,i]*=(-1)        
        elif i==xE3 and lat_in[j,i]==-1:
            lat_in[j,i]=0
            Nbk_in[2]+=1
        else:
            print('Error k=2: substrate reaction not allowed on bridge')    
    
    elif k==3:     # pdt/inter desorption           
        if i in np.arange(xE1,xE2) and lat_in[j,i]==1:
            lat_in[j,i]=0
            Nbk_in[0]+=1   
        elif i in np.arange(xE2,xE3) and lat_in[j,i]==-1:
            lat_in[j,i]=0
            Nbk_in[1]+=1
        elif i==xE3:
            print('Error k=3, no pdt desorption on E3 site, that is involved in the reaction')   
        else:
            print('Error k=3,desorption on a wrong site')
    
    elif k==4:    # right hop
        if i in np.arange(xE1,xE2) and lat_in[j,i]==1:
            lat_in[j,i], lat_in[j,i+1] = lat_in[j,i+1], lat_in[j,i] 
        elif i in np.arange(xE2,xE3) and lat_in[j,i]==-1:
            lat_in[j,i], lat_in[j,i+1] = lat_in[j,i+1], lat_in[j,i] 
        elif i==xE3:
            print('Error k=4, no hop-right on last site')   
        else:
            print('Error k=4, right hop event on wrong site')
    
    elif k==5:  # left hop
        if i in np.arange(xE1+2,xE2) and lat_in[j,i]==1:
            lat_in[j,i], lat_in[j,i-1] = lat_in[j,i-1], lat_in[j,i]
        elif i in np.arange(xE2+2,xE3) and lat_in[j,i]==-1:
            lat_in[j,i], lat_in[j,i-1] = lat_in[j,i-1], lat_in[j,i]
        elif i==xE3:
            print('Error k=5, no hop-rleft on last site')   
        else:
            print('Error k=5, left hop event on wrong site')    
    
    else:
        print('Need to input the event index k between 0 and 5')
        
    return lat_in,Nbk_in  

In [ ]:
np.arange(xE1+2,xE2-1)

#### quick test on func_ext
lat_test = np.random.randint(3, size=(1, 11))-1
print(lat_test)
Nbk_test=[100,30,10]
print(Nbk_test)

lat_test,Nbk_test=func_ext(lat_in=lat_test,Nbk_in=Nbk_test,index=(4,0,2))
print(lat_test)
print(Nbk_test)

new_lat_test,new_Nbk = func_ext(lat_test,Nbk_test,(2,3,0))
print(new_lat_test)
print(new_Nbk)

### Lattice matrix generation function: *<font color=blue>func__lattice_init</font>*

In [ ]:
def func_lattice_init():
    # Nhop is the number of hopping sites on each enzyme couple
    # Nprl is the number of parallel enzyme couple used in this simulation
    return np.zeros(shape=(prl,dim), dtype=np.int); # make an empty surface

print('here is an example')
print(func_lattice_init().shape)

## KMC Loop

In [ ]:
import time
from tqdm import tqdm_notebook as tqdm

dl=[]

for kk in np.arange(0,rep): 
    lat_mat = func_lattice_init()
    t_0 = 0
    Nbk_0 = np.array([0,0,0]) # product from [E1,E2,E3]
    
    R_mat=copy.deepcopy(RC_mat)
    R_mat[0][1:(xE2+1)] = RC_mat[0][1:(xE2+1)]*Nbk_0[0] # involve the bulk concentration to bridge and active sites
    R_mat[0][(xE2+1):]  = RC_mat[0][(xE2+1):]*Nbk_0[1] 

    gamma_mat = np.zeros((6, prl,dim))
    gamma_mat = func_gamma(lat_in   = lat_mat,
                           conc_in  = (Nbk_0/sc.N_A/vol),
                           gamma_in = gamma_mat,
                           index    = (0,0,0),
                           scan     = 'whole') # for the first time, the whole lattice will be scanned to calculate the rate
    gamma_shape = gamma_mat.shape
    
    d={'Nbk':        [Nbk_0],
       't':          [t_0],
       'event_index':[]
      }
    
    for ii in tqdm(range(1,st+1)):
        if d['t'][-1]>=1000:
            break
            
        Rho_1, Rho_2= 1-np.random.rand(2) # use '1-' to make the range from [0,1) to (0,1]
        cum_gamma_mat = np.cumsum(gamma_mat) # 1D cumulative result
        event_index   = np.unravel_index(np.argmax(cum_gamma_mat >= Rho_1*cum_gamma_mat[-1]),gamma_shape) # Select event and convert to 3D index
        #d['Rho'].append([Rho_1, Rho_2])
        
        delta_t = d['t'][-1] - np.log(Rho_2)/cum_gamma_mat[-1]
        d['t'].append(delta_t) # calculate dt and append it
        
        d['event_index'].append(event_index)
        
        lat_mat, Nbk = func_ext(lat_in = lat_mat,
                                Nbk_in = d['Nbk'][-1],
                                index  = event_index)# execute the event

        d['Nbk'].append(Nbk)
        
        gamma_mat = func_gamma(lat_in   = lat_mat, #cascade lattice surface
                               conc_in  = (d['Nbk'][-1]/sc.N_A/vol), # bulk [I] concentration in mol/L
                               gamma_in = gamma_mat,
                               index    = event_index,
                               scan     = 'vicinity') #calculate the rates for next step 
    
    dl.append(d)
    print ('Run-%i: time is %g s, step is %i' %(len(dl),d['t'][-1],len(d['t'])))

### Make a movie  
This is used to check if there is some significant issue with the KM Csimulation

import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.animation as animation

mpl.rcParams['figure.figsize']=0.5*np.array([prl, (lat_hop+2)])

#### http://stackoverflow.com/questions/10429556/animate-quadratic-grid-changes-matshow
fig, ax = plt.subplots()

lat_mat_tran = dl[0]['lat_mat'][0:50]
mat = ax.matshow(lat_mat_tran[0], vmin=-1, vmax=1)
mov_frm = lat_mat_tran #make the movie every 0.1*st steps
anim = animation.FuncAnimation(fig, mat.set_data, frames=mov_frm)
display_animation(anim, default_mode='once')

#### anim.save('animation-CH-leak.mp4')

### Fitting function: *<font color=blue>func_fit</font>*

In [ ]:
def func_fit(x,y,per1,per2):
    slop, y_intercept=np.polyfit(x[int(len(x)*per1):int(len(x)*per2)],y[int(len(y)*per1):int(len(y)*per2)],1) 
    xx=np.arange(x[0],x[-1])
    yy=xx*slop+y_intercept
    xx=xx[np.where((yy>=0)&(yy<=max(yy)))]
    yy=yy[np.where((yy>=0)&(yy<=max(yy)))]
    lag=xx[np.where(yy==min(abs(yy)))]
    xx=xx[(len(xx)*np.arange(0,1,0.01)).astype(int)]
    yy=yy[(len(yy)*np.arange(0,1,0.01)).astype(int)]
    return xx, yy, lag, slop, y_intercept

### Post Analysis  
Calculate the things we interested based on the event_index evolution.

In [ ]:
i=0
for d in dl:
    e0 = np.array([dx[0] for dx in d['event_index']]) # event species
    e1 = np.array([dx[1] for dx in d['event_index']]) # cascade index
    e2 = np.array([dx[2] for dx in d['event_index']]) # site index on cascade
    length=len(e0)
    
    # reaction event evolution
    r1=np.zeros(length);  r1[np.where((e0==2) & (e2==xE1))]=1;  r1 = np.cumsum(r1)/sc.N_A/vol*10**6 # in uM
    r2=np.zeros(length);  r2[np.where((e0==2) & (e2==xE2))]=1;  r2 = np.cumsum(r2)/sc.N_A/vol*10**6 # in uM
    r3=np.zeros(length);  r3[np.where((e0==2) & (e2==xE3))]=1;  r3 = np.cumsum(r3)/sc.N_A/vol*10**6 # in uM

    r2_fitx, r2_fity, r2_lag, slop, y_intercept = func_fit(d['t'][1:],r2,0.5,1)    # extend the steady state curve to calculated lagtime    
    r3_fitx, r3_fity, r3_lag, slop, y_intercept = func_fit(d['t'][1:],r3,0.5,1)
    
    lk1=np.zeros(length);    lk1[np.where((e0==3) & (e2==xE1))]=1;    lk1 = np.cumsum(lk1)/sc.N_A/vol*10**6 # in uM
    lk4=np.zeros(length);    lk4[np.where((e0==3) & (e2==xE2))]=1;    lk4 = np.cumsum(lk4)/sc.N_A/vol*10**6 # in uM
    
    lk3=np.zeros(length);    lk3[np.where((e0==3) & (e2==xE2-1))]=1;    lk3 = np.cumsum(lk3)/sc.N_A/vol*10**6 # in uM
    lk6=np.zeros(length);    lk6[np.where((e0==3) & (e2==xE3-1))]=1;    lk6 = np.cumsum(lk6)/sc.N_A/vol*10**6 # in uM    
    
    lk2=np.zeros(length);    lk2[np.where((e0==3) & (e2>xE1) & (e2<xE2-1))]=1;    lk2 = np.cumsum(lk2)/sc.N_A/vol*10**6 # in uM
    lk5=np.zeros(length);    lk5[np.where((e0==3) & (e2>xE2) & (e2<xE3-1))]=1;    lk5 = np.cumsum(lk5)/sc.N_A/vol*10**6 # in uM

    # Dilute list index
    csv_idx=[0]
    j=0
    for x in d['t']:
        if x-d['t'][csv_idx[-1]]>=0.1: # get the points every 0.1 sec
            csv_idx.append(j)
        j+=1
    csv_idx=np.array(csv_idx)
    # update the dictornary from each KMC simulations
    dl[i].update({'r1':r1,'r2':r2,'r3':r3,
                  'r2_fitx':r2_fitx,'r2_fity':r2_fity,'r2_lag':r2_lag,
                  'r3_fitx':r3_fitx,'r3_fity':r3_fity,'r3_lag':r3_lag,
                  'csv_idx':csv_idx,
                  'lk1':lk1,'lk2':lk2,'lk3':lk3,'lk4':lk4,'lk5':lk5,'lk6':lk6
                 })
    i+=1    
dl[0].keys()

### Summary

In [ ]:
r2_lag_list = [d['r2_lag'][0] for d in dl]
r3_lag_list = [d['r3_lag'][0] for d in dl]

avg_r2lag, std_r2lag = np.average(r2_lag_list), np.std(r2_lag_list)
avg_r3lag, std_r3lag = np.average(r3_lag_list), np.std(r3_lag_list)

print('Lag on E2: %.2f +- %.2f s' %(avg_r2lag,std_r2lag))
print('Lag on E3: %.2f +- %.2f s' %(avg_r3lag,std_r3lag))

## Plotting

In [ ]:
mpl.rcParams['figure.figsize']=(8.0, 4.5)
mpl.rcParams['figure.dpi']=100
mpl.rcParams['xtick.direction']='in'
mpl.rcParams['ytick.direction']='in'
mpl.rcParams['xtick.top']='True'
mpl.rcParams['ytick.right']='True'
mpl.rcParams['font.size']=15.0
mpl.rcParams['savefig.bbox']='tight'
mpl.rcParams['savefig.transparent']='True'
mpl.rcParams['axes.titlesize']=24
mpl.rcParams['axes.labelsize']=20
mpl.rcParams['xtick.labelsize']=16
mpl.rcParams['ytick.labelsize']=16

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(10,8))

i=0
for d in [dl[0]]:
    
    t_cut =np.array(d['t'])[d['csv_idx']][1:]
    i_all=np.array(d['Nbk'])
    i1_cut=i_all[:,0][d['csv_idx']][1:]
    i2_cut=i_all[:,1][d['csv_idx']][1:]
    i1_cut=i1_cut/sc.N_A/vol*10**6
    i2_cut=i2_cut/sc.N_A/vol*10**6    
  
    r1_cut=d['r1'][(d['csv_idx']-1)[1:]]
    r2_cut=d['r2'][(d['csv_idx']-1)[1:]]
    r3_cut=d['r3'][(d['csv_idx']-1)[1:]]
    
    lk1_cut=d['lk1'][(d['csv_idx']-1)[1:]]
    lk2_cut=d['lk2'][(d['csv_idx']-1)[1:]]
    lk3_cut=d['lk3'][(d['csv_idx']-1)[1:]]
    lk4_cut=d['lk4'][(d['csv_idx']-1)[1:]]
    lk5_cut=d['lk5'][(d['csv_idx']-1)[1:]]
    lk6_cut=d['lk6'][(d['csv_idx']-1)[1:]]
        
    axs[0,0].plot(t_cut,r1_cut,linewidth=3,label='[P] from E1')
    axs[0,0].plot(t_cut,r2_cut,linewidth=3,label='[P] from E2')
    axs[0,0].plot(t_cut,r3_cut,linewidth=3,label='[P] from E3')   
    axs[0,0].plot(d['r2_fitx'],d['r2_fity'],'--',linewidth=1.5,label='fit E2')
    axs[0,0].plot(d['r3_fitx'],d['r3_fity'],'--',linewidth=1.5,label='fit E3')
    axs[0,0].set_ylabel('Reaction/ $\mu M$',fontsize=18)
    axs[0,0].set_xlabel('Time / sec',fontsize=18)
    axs[0,0].legend()

    axs[0,1].plot(t_cut,i1_cut,linewidth=3,label='[I-1]')
    axs[0,1].plot(t_cut,i2_cut,linewidth=3,label='[I-2]')
    axs[0,1].set_xlabel('Time / sec',fontsize=18)
    axs[0,1].set_ylabel('Bulk [I]/ $\mu M$',fontsize=18)
    axs[0,1].legend()
    
    axs[1,0].plot(t_cut,lk1_cut,linewidth=3,label='E1 to brg')
    axs[1,0].plot(t_cut,lk2_cut,linewidth=3,label='on brg')
    axs[1,0].plot(t_cut,lk3_cut,linewidth=3,label='brg to E2')
    axs[1,0].plot(t_cut,lk4_cut,linewidth=3,label='E2 to brg')
    axs[1,0].plot(t_cut,lk5_cut,linewidth=3,label='on brg')
    axs[1,0].plot(t_cut,lk6_cut,linewidth=3,label='brg to E3')
    
    axs[1,0].set_ylabel('[Leak]/ $\mu M$',fontsize=18)
    axs[1,0].set_xlabel('Time / sec',fontsize=18)
    axs[1,0].legend()
    
    
    
    '''
    axs[1,1].plot(dl[0]['t'][1:],dl[0]['c_leak'],linewidth=3,label='[bridge leak]')
    axs[1,1].set_ylabel('Bulk [I]/ $\mu M$',fontsize=18)
    axs[1,1].legend()
    '''
plt.tight_layout()
plt.show()

### Split the product profile into 'Nseg' segment and calculate the rate be fitting slop of each segment

#Nseg=int(max(t)/100)
Nseg=100
grad_t=np.zeros(Nseg)
grad_N_prt=np.zeros(Nseg)
for i in np.arange(Nseg):    
    grad_N_prt[i]=fit(t,N_prt,i*(1/Nseg),(i+1)*(1/Nseg))[3]
    grad_t[i]=max(t)*i*(1/Nseg)

### Write to csv files

cd "/Users/yuanchao/notebooks/KMC-CH"

In [ ]:
import csv
import pandas as pd

df1 = pd.DataFrame({"t_cut" :t_cut,
                    "r1_cut": r1_cut,
                    "r2_cut": r2_cut,
                    "r3_cut": r3_cut,
                    "i1_cut": i1_cut,
                    "i2_cut": i2_cut
                   })

df2 = pd.DataFrame({"r2_fitx": r2_fitx,
                    "r2_fity": r2_fity,
                    "r3_fitx": r3_fitx,
                    "r3_fity": r3_fity,
                   })

df3 = pd.DataFrame({"avg_lag": [avg_r2lag],
                    "std_lag": [std_r2lag],
                    "avg_r3lag": [avg_r3lag],
                    "std_r3lag": [std_r3lag]
                   })
df12 = pd.concat([df1,df2],axis=1)
#df.to_csv(str(samp)+".csv",float_format='%15.9f', sep=" ",quoting=csv.QUOTE_NONE, escapechar=" ",index=False)
#df1.to_csv(str(samp)+"_tCourse.csv",sep=" ",index=False)

df12.to_csv(str(samp)+"_timecourse.csv",sep=" ",index=False)

df3.to_csv(str(samp)+"_summary.csv",sep=" ",index=False)
df12.head(n=5)

In [ ]:
df3.head(n=15)

### Physical Constant  
sc.k: boltzman constant in 1.38064852e-23 J/K;  
sc.R: gas constant in 8.3144598 J/(K mol);  
sc.N_A: avgardo number in 6.022140857e+23 mol-1;   